In [1]:
import pandas as pd
import numpy as np
import os

import emoji

import matplotlib.pyplot as plt

import urllib.request

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [21]:
import konlpy

from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [25]:
print(kkma.nouns('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [6]:
file = r"https://raw.githubusercontent.com/reejungkim/WebCrawler/master/GooglePlayStore_Review_2021-02-14-15%3A02%3A28.csv?token=AKLGOYHDODQ5XGME4UC7JTDAFFGXC"

In [7]:
df = pd.read_csv(file)

In [8]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55


In [9]:
df.shape

(200, 6)

In [10]:
def language_preprocessor(string):
    text =  re.sub('[^a-zA-Z]', ' ', string) #replace non-english with blank
    text = text.lower().split() #decapitalize texts
    text = [word for word in text if not word in stops] #remove stopword
    text = [lm.lemmatize(word) for word in text]  #lemmatization of texts
    return text #' '.join(text)
    

In [18]:
import re

def clean_text(texts):
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        corpus.append(review)
    return corpus

df['text processed'] = df['review'].apply(lambda x: clean_text(x)) 

In [26]:
df['nouns'] = df['review'].apply(lambda x: kkma.nouns(x))

In [27]:
df.head()

,Unnamed: 0,author,developer_reply,review,review_date,reviewer_ratings,text processed,nouns
0,0,임닉이,NaN,회원가입을하려니까 가입 필수조건을 다 체크하고 전화번호 인증하니까 필수조건 체크가...,2021년 2월 8일,52,"[, 회, 원, 가, 입, 을, 하, 려, 니, 까, , 가, 입, , 필, 수, ...","[회원, 가입, 하려, 필수, 필수조건, 조건, 체크, 전화, 전화번호, 번호, 인..."
1,1,Nayeon kang,NaN,나름 대기업인데 일좀 열심히 했으면 좋겠다. 쇼핑앱 여러개 사용중인데 이렇게 뭐 ...,2021년 2월 5일,51,"[, 나, 름, , 대, 기, 업, 인, 데, , 일, 좀, , 열, 심, 히, ,...","[나름, 대기업, 일, 쇼핑, 쇼핑앱, 앱, 개, 사용, 사용중, 중, 뭐, 이벤트..."
2,2,산뜻산뜻,NaN,주문조회에서 택배&12개월로 보고 싶은데 택배배송을 누르면 12개월이 3개월로 바...,2021년 1월 25일,51,"[, 주, 문, 조, 회, 에, 서, , 택, 배, &, , , 개, 월, 로, ,...","[주문, 주문조회, 조회, 택배, 12, 12개월, 개월, 택배배송, 배송, 3, ..."
3,3,신종철,NaN,시스템 잘도 만들어놨네요.왠만해서 귀찮아서 글도 잘 안쓰는 사람인데요. 시스템 왜...,2021년 1월 20일,51,"[, 시, 스, 템, , 잘, 도, , 만, 들, 어, 놨, 네, 요, ., 왠, ...","[시스템, 글, 사람, 식, 이해, 상담, 상담사, 사, 기본, 기본적인, 적인, ..."
4,4,김재규,"<div class=""LVQB0b""><div class=""QoPmEb""></div>...","장바구니에 있는 목록들의 순서를 이동, 변경할 수 있게 해주면 좋겠습니다. 장바구...",2020년 12월 15일,55,"[, 장, 바, 구, 니, 에, , 있, 는, , 목, 록, 들, 의, , 순, 서...","[장, 장바구니, 바구니, 목록, 순서, 이동, 변경, 수, 시간, 물품, 아래, ..."
